In [1]:
import os
import numpy as np
import cv2
from sklearn.metrics import accuracy_score

In [2]:
def get_masks(groundtruth_masks_path, predicted_masks_path, groundtruth_masks_list):
    groundtruth_labels = list()
    predicted_labels = list()
    
    for mask in groundtruth_masks_list:
        temp_gt_mask = cv2.imread(os.path.join(groundtruth_masks_path, mask.split('.')[0] + '_L.png'))
        temp_pred_mask = cv2.imread(os.path.join(predicted_masks_path, 'mask_' + mask))
        
        temp_gt_label = np.zeros((480, 640, 1), dtype = np.int16)
        temp_pred_label = np.zeros((480, 640, 1), dtype = np.int16)
        
        temp_gt_label[:, :, 0] = 1 * np.all((temp_gt_mask[:, :] == [128, 64, 128]), axis = 2)
        temp_pred_label[:, :, 0] = 1 * np.all((temp_pred_mask[:, :] == [0, 255, 0]), axis = 2)
        
        groundtruth_labels.append(temp_gt_label)
        predicted_labels.append(temp_pred_label)
        
    groundtruth_labels = np.array(groundtruth_labels).reshape(-1)
    predicted_labels = np.array(predicted_labels).reshape(-1)
    
    return groundtruth_labels, predicted_labels

In [3]:
model_dir = 'model_fcn32_ce_250'

train_files_path = '/home/abhishek/Desktop/datasets/camvid/inputs_resized/'
groundtruth_masks_path = '/home/abhishek/Desktop/datasets/camvid/masks_resized/'
predicted_masks_path = os.path.join(os.getcwd(), os.path.join(model_dir, 'masks'))

all_list = os.listdir(train_files_path)

train_list = list(np.load(os.path.join(model_dir, 'train_list.npy')))
valid_list = list(np.load(os.path.join(model_dir, 'valid_list.npy')))
test_list = list(set(all_list).difference(set(train_list + valid_list)))

In [4]:
train_groundtruth_labels, train_predicted_labels = get_masks(groundtruth_masks_path, predicted_masks_path, train_list)
valid_groundtruth_labels, valid_predicted_labels = get_masks(groundtruth_masks_path, predicted_masks_path, valid_list)
test_groundtruth_labels, test_predicted_labels = get_masks(groundtruth_masks_path, predicted_masks_path, test_list)

In [5]:
len(all_list), len(train_list), len(valid_list), len(test_list)

(701, 336, 14, 351)

In [6]:
print('Training Accuracy : ' + str(accuracy_score(train_groundtruth_labels, train_predicted_labels)))

Training Accuracy : 0.9716627042255704


In [7]:
print('Validation Accuracy : ' + str(accuracy_score(valid_groundtruth_labels, valid_predicted_labels)))

Validation Accuracy : 0.9554740978422619


In [8]:
print('Test Accuracy : ' + str(accuracy_score(test_groundtruth_labels, test_predicted_labels)))

Test Accuracy : 0.9605652191654795
